In [1]:
import pandas as pd
import numpy as np

In [2]:
def ratio_norm(row):
    if row["ratio"] > 0.75:
        return "1"
    if row["ratio"] > 0.50 and row["ratio"] <= 0.75:
        return "tres cuartos"
    if row["ratio"] > 0.25 and row["ratio"] <= 0.50:
        return "una mitad"
    else:
        return 0

In [24]:
df = pd.read_excel("Mobiliario.xlsx", sheet_name = "L01", header = None, engine='openpyxl')
mob_df = pd.read_excel("Mobiliario_Vols.xlsx", sheet_name = "L01", engine='openpyxl')
roomsdf = pd.read_excel("Habitaciones.xlsx", sheet_name = "Habitaciones L01", header = 0, engine='openpyxl')
habitacion_dict = {}

In [29]:
for room in range(df.shape[0]):
    room_df = df.iloc[room]
    merged_df = pd.merge(mob_df, room_df, left_index = True, right_index = True)
    merged_df.rename(columns = {room: "Volumen Intersectado"}, inplace = True)
    merged_df['ratio'] = merged_df['Volumen Intersectado'] / merged_df['Volumen']
    filter_mob = merged_df['ratio'] > 0.2
    final_merged_df = merged_df[filter_mob]
    
    if final_merged_df.shape[0] != 0:

        final_merged_df['ratio_norm'] = final_merged_df.apply(lambda row: ratio_norm(row), axis = 1)

        nombres = []
        ratios = []
        cantidades = []
        texto = []
        texto.append("Esta habitación tiene,")

        for i in range(len(final_merged_df.groupby(["Tipo de Mobiliario", "ratio_norm"]).size().index)):
            name = final_merged_df.groupby(["Tipo de Mobiliario", "ratio_norm"]).size().index[i][0]
            ratio = final_merged_df.groupby(["Tipo de Mobiliario", "ratio_norm"]).size().index[i][1]
            amount = final_merged_df.groupby(["Tipo de Mobiliario", "ratio_norm"]).size()[i]
            nombres.append(name)
            ratios.append(ratio)
            cantidades.append(amount)

        for i in range(len(final_merged_df.groupby(["Tipo de Mobiliario", "ratio_norm"]).size().index)):
            texto.append(f"{cantidades[i]} unidades de {nombres[i]} con una ocupación aproximada de {ratios[i]} por unidad")

        texto_final = " ".join(texto)
    else:
        texto_final = "Esta habitación no posee mobiliario"
    
    habitacion_dict[roomsdf.iloc[room]['Id']] = texto_final

<ipython-input-29-f89766aafcc8>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_merged_df['ratio_norm'] = final_merged_df.apply(lambda row: ratio_norm(row), axis = 1)


In [26]:
habitacion_dict

{1935077: 'Esta habitación tiene, 6 unidades de Chair con una ocupación aproximada de 1 por unidad 1 unidades de Table con una ocupación aproximada de 1 por unidad 1 unidades de Wardrobe con una ocupación aproximada de 1 por unidad',
 1935080: 'Esta habitación tiene, 6 unidades de Chair con una ocupación aproximada de 1 por unidad 1 unidades de Table con una ocupación aproximada de 1 por unidad 1 unidades de Wardrobe con una ocupación aproximada de 1 por unidad',
 1935083: 'Esta habitación tiene, 6 unidades de Chair con una ocupación aproximada de 1 por unidad 1 unidades de Table con una ocupación aproximada de 1 por unidad 1 unidades de Wardrobe con una ocupación aproximada de 1 por unidad',
 1935086: 'Esta habitación tiene, 6 unidades de Chair con una ocupación aproximada de 1 por unidad 1 unidades de Table con una ocupación aproximada de 1 por unidad 1 unidades de Wardrobe con una ocupación aproximada de 1 por unidad',
 1935089: 'Esta habitación tiene, 14 unidades de Chair con una o

In [22]:
roomsdf

,Id,Numero,Nombre
0,559029,-1002,Sala de Formación
1,559038,-1005,Sala de RAACS
2,559395,-1007,Sala de Control


In [5]:
roomsdf['Mobiliario'] = roomsdf['Id'].map(habitacion_dict)
final_roomsdf = roomsdf[['Id', 'Mobiliario']]

In [6]:
final_roomsdf

,Id,Mobiliario
0,559029,"Esta habitación tiene, 13 unidades de Chair co..."
1,559038,Esta habitación no posee mobiliario
2,559395,"Esta habitación tiene, 1 unidades de Table con..."


#final_roomsdf.to_excel("HabitacionesFinales.xlsx", sheet_name = "L01", index = False)
with pd.ExcelWriter("HabitacionesFinales.xlsx", engine = 'openpyxl', mode = 'a') as writer:
    final_roomsdf.to_excel(writer, sheet_name = 'B01', index = False)